In [ ]:
!pip install spotipy

In [ ]:
!pip install fa2

In [ ]:
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import pprint
import csv
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
import networkx as nx
from fa2 import ForceAtlas2

In [ ]:
!pip install azlyrics

In [ ]:
!pip install lyricwikia

In [ ]:
!pip install BeautifulSoup4

In [ ]:
from azlyrics import lyrics
import pandas as pd
import re
import lyricwikia
import urllib2
import json
import bs4
import requests
import unicodedata
import nltk

In [ ]:
mkdir lyrics

In [ ]:
mkdir topsters

In [ ]:
#SETTING THE DATAFRAMES WE NEED HERE
artists_df = pd.read_csv('artists.csv')

In [ ]:
client_id = "b06999e849764d0cb9d85ff2b4762fd9"
client_secret = "4565b9044d694deb9cf54eddb9b08e69"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
len(artists_df.index)

In [ ]:
artist_dict = {}
counter = 0
for index, row in artists_df.iterrows():
    artist_id = row['Artist_id']
    artist_name = row['Artist']
    if (artist_id in artist_dict):
        print(artist_id)
        print(artist_name)
        print(artist_dict[artist_id])
        counter += 1
        continue
    artist_dict[artist_id] = artist_name
print(counter)

In [ ]:
counter = 0
counter_dup = 0
import os.path
for index, row in artists_df.iterrows():
    counter += 1
    if (counter % 10 == 0):
        print(counter)
    artist_id = row['Artist_id']
    artist_name = row['Artist']
    try:
        if (os.path.exists("./topsters/songs_"+str(artist_id)+".csv")):
            print(artist_id)
            counter_dup += 1
        fileCsv2 = csv.writer(open("./topsters/songs_"+str(artist_id)+".csv", "w+"))
        fileCsv2.writerow(['Artist', 'Artist_id', 'Song', 'Song_id'])
        top_track_list = sp.artist_top_tracks(str(artist_id))['tracks']
    except:
        print(artist_id)
        pass
    for track in top_track_list:
        row = [artist_name.encode('ascii', 'ignore'),str(artist_id).encode('ascii', 'ignore'),track['name'].encode('ascii', 'ignore'), track['id'].encode('ascii', 'ignore')]
        fileCsv2.writerow(row)
print(counter_dup)

In [ ]:
genius_api_dict = {'id':'eOFdp-v__I7JeKporMF3K49wKnz3cYIkJcOGUP1wRz2uV7AmWh1eeBup_zJMCkqA',
              'secret': 'kbbVWQxhXtsfnHm1W65gkjnvxlt9A3U2ih5bf6KWBJe6_KeYWZWyxC4rDK62OX9OQ9cLWTH0FgKZWoRmEofH9g',
              'access_token': 'jhpqKVzO880gvQ5i-JkEh08wTUhmmA4kMuCkt5MQKLKMrWnjjqs3Z0WiVd49TmR7'}

In [ ]:
def genius_Lyrics(artist, song):
    querystring = "http://api.genius.com/search?q=" + urllib2.quote(song + " " + artist)
    request = urllib2.Request(querystring)
    request.add_header("Authorization", "Bearer " + genius_api_dict['access_token'])   
    request.add_header("User-Agent", "curl/7.9.8 (i686-pc-linux-gnu) libcurl 7.9.8 (OpenSSL 0.9.6b) (ipv6 enabled)") #Must include user agent of some sort, otherwise 403 returned
    while True:
        try:
            response = urllib2.urlopen(request, timeout=4) #timeout set to 4 seconds; automatically retries if times out
            raw = response.read()
        except socket.timeout:
            print("Timeout raised and caught")
            continue
        break
    json_obj = json.loads(raw)
    body = json_obj["response"]["hits"]

    num_hits = len(body)
    if num_hits==0:
        print("\t\tNo results for: " + song)      

    for result in body:

#             result_id = result["result"]["id"]
#             title = result["result"]["title"]
        url = result["result"]["url"]
#             path = result["result"]["path"]
#             header_image_url = result["result"]["header_image_url"]
#             annotation_count = result["result"]["annotation_count"]
#             pyongs_count = result["result"]["pyongs_count"]
#             primaryartist_id = result["result"]["primary_artist"]["id"]
#             primaryartist_name = result["result"]["primary_artist"]["name"]
#             primaryartist_url = result["result"]["primary_artist"]["url"]
#             primaryartist_imageurl = result["result"]["primary_artist"]["image_url"]
#             row=[page,result_id,title,url,path,header_image_url,annotation_count,pyongs_count,primaryartist_id,primaryartist_name,primaryartist_url,primaryartist_imageurl]
#             print(row) #write as CSV
        page = requests.get(url)
        if page.status_code == 404:
            return None

        # Scrape the song lyrics from the HTML
        html = bs4.BeautifulSoup(page.text, "html.parser")
        title_genius = html.find("h1", class_="header_with_cover_art-primary_info-title").get_text().strip()
        artist_genius = html.find("a", class_="header_with_cover_art-primary_info-primary_artist").get_text().strip()
#         print('artist_genius: {} - artist: {}'.format(artist_genius, artist))
        if (title_genius == song.strip() ):#and artist_genius == artist):
            lyrics = html.find("div", class_="lyrics").get_text()
            lyrics = re.sub('(\[.*?\])*', '', lyrics)
            lyrics = re.sub('\n{2}', '\n', lyrics)  # Gaps between verses

            lyrics = unicodedata.normalize('NFKD', lyrics).encode('ascii','ignore')
            return lyrics.strip('\n')
    return None    
        

In [ ]:
def removePunctuation(tokens):
    """
    This function finds punctuation (apart from '#' symbols) from each string token inside the provided token list
    
    arguments: tokens: list of strings
    returns: list of strings (tokens)
    """
    new_token_list = []
    for token in tokens:
        #returns a new string without punctuation
        new_token = re.sub(r'[^\w\s#]','',token)
        if (len(new_token) > 0):
            new_token_list.append(new_token)
    return new_token_list

In [ ]:
for index, row in artists_df.iterrows():
    artist_id = row['Artist_id']
    artist_name = row['Artist']
    artists_song_df = pd.read_csv('./songs/songs_'+artist_id+'.csv')
    fileCsv_song_lyrics = csv.writer(open('./lyrics/lyrics_'+artist_id+'.csv', "w+"))
    fileCsv_song_lyrics.writerow(['Artist', 'Artist_id','Song','Song_id','Lyrics'])

#     with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#         print(artists_song_df)
#     break
    unique_song_list = []
    for index_song, row_song in artists_song_df.iterrows():
#         print(row_song['Song'])
        if (not(row_song['Song'] in unique_song_list)):
            unique_song_list.append(row_song['Song'])
            lyrics = None
            if (not('strumental' in row_song['Song'] and 'emix' in row_song['Song'])):
                song_name_wo_parentheses = re.sub(r'\([^()]*\)', '', row_song['Song'])
                print artist_name
                print "===="+song_name_wo_parentheses
                try:
                    wd = lyrics(artist_name, song_name_wo_parentheses)
                    if ('Error' in wd):
                        wd = lyrics(artist_name, row_song['Song'])
                    if ('Error' in wd):
                        raise Exception
            #             print(song_name_wo_parentheses +' - Unable to find lyrics in AZ')
                    else:
                        lyrics = wd
                        print(row_song['Song'] +' - FOUND LYRICS in AZ')

                except: #if shadow banned from AZLyrics
                    print('\tAZ failed')
                    try:
                        lyrics = lyricwikia.get_lyrics(artist_name, song_name_wo_parentheses)
                    except:
                        print('\tLyric Wikia failed')
                        try:
                            lyrics = genius_Lyrics(artist_name, song_name_wo_parentheses)
                            if (lyrics == None):
                                print("\tGenius failed")
                        except:
                            print('Socket error - Sleeping 5 sec')
                            time.sleep(5)
            if not(lyrics == None):
                #tokenize lyrics
                lyrics = nltk.word_tokenize(lyrics)
                lyrics = removePunctuation(lyrics)
                if isinstance(lyrics, list):
                    lyric_string = ""
                    for lyric in lyrics:
                        lyric_string = lyric_string + " " + lyric
                    lyrics = lyric_string
                fileCsv_song_lyrics.writerow([artist_name, artist_id,row_song['Song'],row_song['Song_id'],lyrics])           #do NOT write in the CSV if you cannot find the lyrics
                print("\tADDED LYRICS")


In [ ]:
result = sp.artist_albums('spotify:artist:5pKCCKE2ajJHZ9KAiaK11H')
for album in result['items']:
  print(album['uri'])
  result1 = sp.album_tracks(album['uri'])
    #pprint.pprint(result1)
  for song in result1['items']:
      pprint.pprint(song['name'])

In [ ]:
result = sp.artist_top_tracks('spotify:artist:5pKCCKE2ajJHZ9KAiaK11H')
#pprint.pprint(result)
for album in result['tracks']:
  #pprint.pprint(album['album']['uri'])
  result1 = sp.album_tracks(album['album']['uri'])
  #pprint.pprint(result1)
  for song in result1['items']:
      pprint.pprint(song['name'])
      wd = lyrics("Rihanna", song['name'])
      for line in wd:
        print(line)
      break